In [1]:
import pandas as pd
import pickle
from tqdm.auto import tqdm
from collections import defaultdict
from joblib import Parallel, delayed

import torch

%load_ext autoreload
%autoreload 2
from pywikidata import Entity

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_df = pd.read_csv("./wdsq_dataset/train_no_prefix_tree.csv")
valid_df = pd.read_csv("./wdsq_dataset/valid_no_prefix_tree.csv")

answer_cols = [c for c in train_df.columns if "answer_" in c]

In [3]:
def _proc_row(row):
    candidates = [e for _, e in Entity(row["S"]).forward_one_hop_neighbors]
    for lbl in row[answer_cols].unique():
        try:
            candidates.append(Entity.from_label(str(lbl))[0])
        except ValueError:
            pass
    candidates = list(set(candidates))

    target = Entity(row["O"])

    items = []
    for candidate in candidates:
        item = {}
        item["question"] = row["Q"]
        item["answer"] = candidate.idx
        item["target"] = target.idx
        item["is_correct"] = candidate == target
        items.append(item)
    return items

In [4]:
train_dataset = Parallel(n_jobs=6)(
    delayed(_proc_row)(row)
    for _, row in tqdm(train_df.iterrows(), total=train_df.index.size)
)

valid_dataset = Parallel(n_jobs=6)(
    delayed(_proc_row)(row)
    for _, row in tqdm(valid_df.iterrows(), total=valid_df.index.size)
)

  2%|▏         | 390/19481 [00:01<01:15, 254.07it/s]

2023-01-10 12:56:56,769 [ERROR]: {"msg": "Expecting value: line 1 column 1 (char 0)", "params": {"format": "json", "query": "\n        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> \n        SELECT * WHERE{\n            ?item rdfs:label \"William \"Bill\" Bingham\"@en .\n        }\n        "}, "endpoint": "https://query.wikidata.org/sparql", "response": {"status_code": 400, "headers": {"server": "nginx/1.14.2", "date": "Tue, 10 Jan 2023 12:56:56 GMT", "content-type": "text/plain", "x-served-by": "wdqs1012", "access-control-allow-origin": "*", "content-encoding": "gzip", "vary": "Accept-Encoding", "age": "0", "x-cache": "cp3064 miss, cp3052 pass", "x-cache-status": "pass", "server-timing": "cache;desc=\"pass\", host;desc=\"cp3052\"", "strict-transport-security": "max-age=106384710; includeSubDomains; preload", "report-to": "{ \"group\": \"wm_nel\", \"max_age\": 86400, \"endpoints\": [{ \"url\": \"https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error

  2%|▏         | 486/19481 [00:02<01:14, 256.36it/s]

2023-01-10 12:56:57,477 [ERROR]: {"msg": "Expecting value: line 1 column 1 (char 0)", "params": {"format": "json", "query": "\n        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> \n        SELECT * WHERE{\n            ?item rdfs:label \"William \"Bill\" Smith\"@en .\n        }\n        "}, "endpoint": "https://query.wikidata.org/sparql", "response": {"status_code": 400, "headers": {"server": "nginx/1.14.2", "date": "Tue, 10 Jan 2023 12:56:57 GMT", "content-type": "text/plain", "x-served-by": "wdqs1007", "access-control-allow-origin": "*", "content-encoding": "gzip", "vary": "Accept-Encoding", "age": "1", "x-cache": "cp3050 miss, cp3052 pass", "x-cache-status": "pass", "server-timing": "cache;desc=\"pass\", host;desc=\"cp3052\"", "strict-transport-security": "max-age=106384710; includeSubDomains; preload", "report-to": "{ \"group\": \"wm_nel\", \"max_age\": 86400, \"endpoints\": [{ \"url\": \"https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&s

  3%|▎         | 582/19481 [00:02<01:19, 237.91it/s]

...

2023-01-10 12:57:58,625 [ERROR]: {"msg": "Expecting value: line 1 column 1 (char 0)", "params": {"format": "json", "query": "\n        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> \n        SELECT * WHERE{\n            ?item rdfs:label \"William \"Bill\" Cox\"@en .\n        }\n        "}, "endpoint": "https://query.wikidata.org/sparql", "response": {"status_code": 400, "headers": {"server": "nginx/1.14.2", "date": "Tue, 10 Jan 2023 12:57:58 GMT", "content-type": "text/plain", "x-served-by": "wdqs1007", "access-control-allow-origin": "*", "content-encoding": "gzip", "vary": "Accept-Encoding", "age": "0", "x-cache": "cp3064 miss, cp3052 pass", "x-cache-status": "pass", "server-timing": "cache;desc=\"pass\", host;desc=\"cp3052\"", "strict-transport-security": "max-age=106384710; includeSubDomains; preload", "report-to": "{ \"group\": \"wm_nel\", \"max_age\": 86400, \"endpoints\": [{ \"url\": \"https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&sch

 95%|█████████▍| 2676/2821 [00:06<00:00, 361.50it/s]

2023-01-10 12:57:58,899 [ERROR]: {"msg": "Expecting value: line 1 column 1 (char 0)", "params": {"format": "json", "query": "\n        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> \n        SELECT * WHERE{\n            ?item rdfs:label \"William \"Bill\" Brown\"@en .\n        }\n        "}, "endpoint": "https://query.wikidata.org/sparql", "response": {"status_code": 400, "headers": {"server": "nginx/1.14.2", "date": "Tue, 10 Jan 2023 12:57:58 GMT", "content-type": "text/plain", "x-served-by": "wdqs1007", "access-control-allow-origin": "*", "content-encoding": "gzip", "vary": "Accept-Encoding", "age": "0", "x-cache": "cp3056 miss, cp3052 pass", "x-cache-status": "pass", "server-timing": "cache;desc=\"pass\", host;desc=\"cp3052\"", "strict-transport-security": "max-age=106384710; includeSubDomains; preload", "report-to": "{ \"group\": \"wm_nel\", \"max_age\": 86400, \"endpoints\": [{ \"url\": \"https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&s

100%|██████████| 2821/2821 [00:06<00:00, 408.99it/s]


2023-01-10 12:57:59,228 [ERROR]: {"msg": "Expecting value: line 1 column 1 (char 0)", "params": {"format": "json", "query": "\n        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> \n        SELECT * WHERE{\n            ?item rdfs:label \"William \"Bill\" O'Connor\"@en .\n        }\n        "}, "endpoint": "https://query.wikidata.org/sparql", "response": {"status_code": 400, "headers": {"server": "nginx/1.14.2", "date": "Tue, 10 Jan 2023 12:57:59 GMT", "content-type": "text/plain", "x-served-by": "wdqs1016", "access-control-allow-origin": "*", "content-encoding": "gzip", "vary": "Accept-Encoding", "age": "1", "x-cache": "cp3060 miss, cp3052 pass", "x-cache-status": "pass", "server-timing": "cache;desc=\"pass\", host;desc=\"cp3052\"", "strict-transport-security": "max-age=106384710; includeSubDomains; preload", "report-to": "{ \"group\": \"wm_nel\", \"max_age\": 86400, \"endpoints\": [{ \"url\": \"https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_erro

In [5]:
train_df = pd.DataFrame([item for sublist in train_dataset for item in sublist])
valid_df = pd.DataFrame([item for sublist in valid_dataset for item in sublist])

In [7]:
import datasets

train_dataset = datasets.Dataset.from_pandas(train_df)
valid_dataset = datasets.Dataset.from_pandas(valid_df)

In [8]:
train_dataset

Dataset({
    features: ['question', 'answer', 'target', 'is_correct'],
    num_rows: 987668
})

In [6]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer

tokenizer = BertTokenizer.from_pretrained("bert-large-cased")
model = BertForSequenceClassification.from_pretrained("bert-large-cased", num_labels=2)

training_args = TrainingArguments(output_dir="test_trainer")
trainer = Trainer(
    model=model,
    args=training_args,
)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 